## Presentation Type Classifier and Networks


### What can You do with this Notebook

- Find and classify imitative points (fugas, imitative duos, and periodic entries)
- Build a network of patterns found in several pieces, and reveal 'communities' of compositions


#### Network Visualization with Louvain Communities

Read more about **Louvain Community Detection Algorithms**:  https://towardsdatascience.com/community-detection-algorithms-9bd8951e7dae

**Updated Summer 2022**
**Key Features**:

- Uses `getDistance` to identify `close matches` with side-by-side comparison of soggetti.  
- For example: With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same. These are reported as "flexed entries" in a separate column.
- Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
- Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
- Labels Fuga, PEn, and ID according to time intervals between entries. NIm not yet supported!
- If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
- Counts number of entries
- Provides offset and measure/beat locations
- Sorts all presentation types by the order in which they appear in the piece
- Reports voice names of the entries, in order of their appearance
- Omits singleton soggetti (just one entry of a given motive in isolation)
    
- Read the documentation:  `print(piece.presentationTypes.__doc__)`

In [ ]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
import glob
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

## A.  Start by Finding Presentation Types in a Corpus of Pieces 

* The **CorpusBase** class is a convenient way to find patterns in any given list of pieces.
* The pieces are provided as a **list**, within square brackets and separated by commas.  
* The bracketed list is then contained within the parentheses of `CorpusBase()`
* For example: 


```prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'
piece_list = ['CRIM_Model_0019',
             'CRIM_Mass_0019_1',
             'CRIM_Mass_0019_2',
             'CRIM_Mass_0019_3',
             'CRIM_Mass_0019_4',
             'CRIM_Mass_0019_5']
suffix = '.mei'
corpus_list  = []
for piece in piece_list:
url = prefix + piece + suffix
    corpus_list.append(url)

```
Or for local files:

```

corpus_list = []
for name in glob.glob('Music_Files/*'):
    corpus_list.append(name)

```




* Read the documentation:  `print(CorpusBase.batch.__doc__)`



In [ ]:
# remote corpus

corpus_list = []

prefix = 'https://crimproject.org/mei/'

piece_list = ['CRIM_Model_0019', 'CRIM_Mass_0019_1']
suffix = '.mei'

for piece in piece_list:
    url = prefix + piece + suffix
    corpus_list.append(url)


corpus_list

In [ ]:
# local files
corpus_list = []
for name in glob.glob('Music_Files/*'):
    corpus_list.append(name)


### Now Run the PresentationType method on the Corpus

In [ ]:
# This will need to be modified if using a corpus method different from the CRIM Project above.
final = pd.DataFrame()

for work in corpus_list:

    piece = importScore(work)   
    output = piece.presentationTypes(limit_to_entries = True,
                                     head_flex = 1,
                                      body_flex = 0,
                                    include_hidden_types = False,
                                    combine_unisons = True,
                                   melodic_ngram_length = 4)

    final['Validation'] = ""
    final['Comments'] = ""
    final = final.append(output, ignore_index=True)

# final.to_csv("corpus_classified.csv")
# final.to_csv("saved_csv/CRIM_Mass_0019_P_Types_hidden.csv")

In [ ]:
final


## Various Options to Filter to Prepare to Network

See the Presentation Type Filter Notebook for more ideas!

But the results of this filtering are needed to make useful networks below!

* by type
* by piece
* according to length between entries

In [ ]:
# filter for only ID or PEN
pen_id_filter = final['Presentation_Type'].isin(["PEN"])
pen_id_only = final[pen_id_filter]
final = pen_id_only.copy()
len(final)



## B. Network Visualization with Louvain Communities

* This will only make sense for a corpus!

## Add Combined Data for Melodic, Time and Soggetti as "Words" for Network Edges

- We combine the lists of time and melodic intervals of entries as strings (thus a kind of word that uniquely identifies that pattern)
- We do the same for soggetti, combining the melodic intervals themselves as a string
- These are in turn assembled in various combinations:  TINT, MINT, and SOG, also as strings

In [ ]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody_new)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["MINT_SOG"] = final["MINT"] + '_' + final["SOG"]
final["TINT_SOG"] = final["TINT"] + '_' + final["SOG"]
final.head()

### Make Groups, Based on Selected Criteria:

* MINT (the melodic intervals between imitative entries)
* TINT (the time intervals between imitative entries)
* SOG (the soggetti)
* Or any of the combinations:
    * ALL (MINT + TINT + Soggetti)
    * ALL_INT (MINT + TINT)
    * MINT_SOG (MINT + Soggetti)
    * TINT_SOG (TINT + Soggetti)

In [ ]:

output_grouped = final.groupby(['Title'])['MINT'].apply(list).reset_index()
output_grouped


## Group and Find Unique Pairs

- the pairing must be applied to columns that have more than one value, like Title, since these form the edges

In [ ]:
pairs = output_grouped.MINT.apply(lambda x: list(combinations(x, 2)))
pairs2 = pairs.explode().dropna()
unique_pairs = pairs.explode().dropna().unique()
len(unique_pairs)

In [ ]:
# define network functions.  Do not edit!

def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

def create_node_html(node: str, source_df: pd.DataFrame, node_col: str):
    rows = source_df.loc[source_df[node_col] == node].itertuples()
    
    html_lis = []
    
    for r in rows:
        html_lis.append(f"""<li>Composer: {r.Composer}<br>
                                Title: {r.Title}<br>
                                Offset: {r.First_Offset}</li>"""
                       )
        
    html_ul = f"""<ul>{''.join(html_lis)}</ul>"""
        
    return html_ul


def add_nodes_from_edgelist(edge_list: list, 
                               source_df: pd.DataFrame, 
                               graph: nx.Graph,
                               node_col: str):
    
    graph = deepcopy(graph)
    
    node_list = pd.Series(edge_list).apply(pd.Series).stack().unique()
    
    for n in node_list:
        graph.add_node(n, title=create_node_html(n, source_df, node_col))
        
    return graph

def choose_network(df, chosen_word, file_name):
    
    if len(df) < 1:
        print("Sorry, No P Types to display")
    else:
        output_grouped = df.groupby(['Title'])[chosen_word].apply(list).reset_index()
        pairs = output_grouped[chosen_word].apply(lambda x: list(combinations(x, 2)))
        pairs2 = pairs.explode().dropna()
        unique_pairs = pairs.explode().dropna().unique()


        pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="black", font_color="white")
        G = nx.Graph()

        try:
            G = add_nodes_from_edgelist(edge_list=unique_pairs, source_df=final, graph=G, node_col=chosen_word)
        except Exception as e:
            print(e)


        G.add_edges_from(unique_pairs)
        G = add_communities(G)
        pyvis_graph.from_nx(G)
        pyvis_graph.show(file_name)

In [ ]:
# Find your rendered network in the Jupyter Hub file list!

choose_network(final, 'MINT', 'Mod_Mass_Full_Sog_19.html')